In [1]:
import sys
sys.path.append('..')

In [42]:
import xgboost as xgb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from core.dataset import *
from core.utils import *

In [3]:
path = 'D:/Data/hoffmanlab/featureselection/data/'

## Polarization state classification on just CpG

In [7]:
ligands = ['CpG_am']
polarizations = ['', 'i0', 'i3', 'i4', 'ib', 'ig']

In [8]:
data = PolarData(path, ligands, polarizations, merge=True)

In [20]:
model = xgb.XGBClassifier(use_label_encoder=False, tree_method='gpu_hist')

In [21]:
X, labels = data.iloc[:, [i for i in range(984)]].to_numpy(), data.iloc[:, [984]].to_numpy().reshape(-1, )

In [22]:
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, labels, test_size=0.1)

In [23]:
model.fit(X_train, y_train)

[21:40:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=0, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='gpu_hist', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [24]:
pred = model.predict(X_val)

In [25]:
cr = sklearn.metrics.classification_report(pred, y_val, target_names=polarizations) # blank is the unstimulated

In [26]:
print(cr)

              precision    recall  f1-score   support

                   0.63      0.52      0.57       231
          i0       0.44      0.52      0.48        99
          i3       0.23      0.30      0.26        69
          i4       0.47      0.59      0.52        73
          ib       0.56      0.54      0.55       164
          ig       0.66      0.62      0.64       160

    accuracy                           0.53       796
   macro avg       0.50      0.51      0.50       796
weighted avg       0.55      0.53      0.54       796



## Holistic polarization state classification

In [29]:
ligands = ['CpG_am', 'FLA_am', 'FSL_am', 'LPS_am', 'P3K_am', 'PIC_am', 'R84_am', 'TNF_am']
polarizations = ['', 'i0', 'i3', 'i4', 'ib', 'ig']

In [30]:
data = PolarData(path, ligands, polarizations, merge=True)

In [33]:
data = data.iloc[:, [i for i in range(985)]]

In [35]:
X, labels = data.iloc[:, [i for i in range(984)]].to_numpy(), data.iloc[:, [984]].to_numpy().reshape(-1, )

In [36]:
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, labels, test_size=0.1)

In [37]:
model = xgb.XGBClassifier(use_label_encoder=False, tree_method='gpu_hist')

In [38]:
model.fit(X_train, y_train)

[22:16:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=0, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='gpu_hist', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [39]:
pred = model.predict(X_val)

In [40]:
cr = sklearn.metrics.classification_report(pred, y_val, target_names=polarizations)

In [41]:
print(cr)

              precision    recall  f1-score   support

                   0.50      0.40      0.45      1683
          i0       0.35      0.46      0.40       892
          i3       0.36      0.36      0.36       994
          i4       0.46      0.45      0.46       884
          ib       0.47      0.46      0.47      1046
          ig       0.50      0.52      0.51      1307

    accuracy                           0.44      6806
   macro avg       0.44      0.44      0.44      6806
weighted avg       0.45      0.44      0.44      6806

